<a href="https://colab.research.google.com/github/marinebon/HackingLimno2025/blob/main/02_Sample_environmental_data_at_each_occurrence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fetching Occurrence Data from pyobis



In [1]:
!pip install rasterio pandas geopandas matplotlib  requests xarray netCDF4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 49.9 MB/s eta 0:00:00


Importing libraries

In [5]:
import pandas as pd
%matplotlib inline

mount google drive for saving result

In [3]:
from google.colab import drive
drive.mount('/content/drive')


import os
project_dir = '/content/drive/MyDrive/GSoC_SDM_Project'
if not os.path.exists(project_dir):
    os.makedirs(project_dir)

Mounted at /content/drive


In [14]:
# read parquet file of occurrences
# see step 01
df_clean = pd.read_parquet(f'{project_dir}/occurrences.parquet')

Function to prepare data for environmental sampling

In [15]:
from typing import Optional, List
import pandas as pd

def prepare_for_environmental_sampling(
    df: pd.DataFrame,
    species_column: str = 'scientificName',
    lat_column: str = 'decimalLatitude',
    lon_column: str = 'decimalLongitude',
    additional_columns: Optional[List[str]] = None
) -> pd.DataFrame:

    columns = [species_column, lat_column, lon_column]

    if additional_columns:
        columns.extend([col for col in additional_columns if col in df.columns])

    prepared_df = df[columns].copy()

    prepared_df['occurrence_id'] = range(len(prepared_df))

    cols = ['occurrence_id'] + [col for col in prepared_df.columns if col != 'occurrence_id']
    prepared_df = prepared_df[cols]

    return prepared_df

This is just and example for testing the general functions


In [16]:
# TODO: move this to pyOBIS notebok

def save_occurrence_data(
    df: pd.DataFrame,
    filename: str,
    format: str = 'csv'
) -> None:

    if format == 'csv':
        df.to_csv(f"{filename}.csv", index=False)
        print(f"Data saved to {filename}.csv")
    elif format == 'parquet':
        df.to_parquet(f"{filename}.parquet", index=False)
        print(f"Data saved to {filename}.parquet")
    else:
        raise ValueError(f"Unsupported format: {format}")


bringing in environmental data. Functions for Environmental Data Sampling

In [17]:
import rasterio
import xarray as xr
import numpy as np
from typing import Union, List, Tuple, Optional, Dict

In [19]:
def add_environmental_data(
    df: pd.DataFrame,
    raster_path: str,
    column_name: str = 'temperature',
    lat_col: str = 'decimalLatitude',
    lon_col: str = 'decimalLongitude'
) -> pd.DataFrame:

    df_result = df.copy()

    with rasterio.open(raster_path) as src:
        coords = [(row[lon_col], row[lat_col]) for _, row in df.iterrows()]
        sampled_values = list(src.sample(coords))
        values = [val[0] if val[0] != src.nodata else np.nan for val in sampled_values]
        df_result[column_name] = values

    print(f"Added {column_name}: {df_result[column_name].notna().sum()}/{len(df_result)} valid values")
    return df_result

RASTER_PATH = f'{project_dir}/cmems_salinity_44d2_bc4b_70b2_U1749495233099.nc'

df_with_temp = add_environmental_data(df_clean, RASTER_PATH, 'temperature')

print(df_with_temp.head())

Added temperature: 2981/3013 valid values
   decimalLatitude  decimalLongitude occurrenceStatus  date_year  \
0        25.138383        -80.624983                Q     2006.0   
1        24.996280        -80.649250                Q     2006.0   
2        25.248783        -80.403150                Q     2005.0   
3        25.252020        -80.397290                Q     2007.0   
4        25.472200        -80.325950                Q     2005.0   

       date_mid  temperature  
0  1.147133e+12    37.060654  
1  1.160006e+12    37.025467  
2  1.115683e+12    37.066723  
3  1.189469e+12    37.066723  
4  1.129680e+12    37.199909  
